### Conformer(CNN + Transfomer) 모델을 이용해 영아 울음소리 분류 모델을 만들어보자.

Conformer 논문: https://arxiv.org/pdf/2005.08100.pdf

In [24]:
import numpy as np
import pandas as pd
import tensorflow as tf
import librosa
import sys
from keras.layers import Input, Dense, Lambda, Flatten, Reshape, Activation, Dropout, Add, TimeDistributed, Multiply, Conv1D, Conv2D, MaxPooling1D, AveragePooling1D
from keras.models import Model, Sequential, load_model
from keras import backend as K
from keras import metrics
from keras import optimizers
from keras.callbacks import History, ModelCheckpoint

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [10]:
sys.path.append('/Users/jaewone/developer/tensorflow/baby-cry-classification')

from utils.sound import *
from utils.os import *
from constant.os import *

In [14]:
sample_data_path = os.path.join(main_path, 'sample_data')
info_csv_path = os.path.join(main_path, 'sample_data.csv')

In [25]:
# 음성 파일 경로와 클래스 레이블 지정
df = pd.read_csv(info_csv_path, index_col=0)
audio_files = [os.path.join(sample_data_path, file) for file in df['file']]
class_labels = df['state'].tolist()

In [26]:
# 멜 스펙트럼과 클래스 레이블 추출
def compute_mel_spectrogram(audio_path, n_fft=2048, hop_length=512, n_mels=128):
    """
    멜 스펙트럼을 계산하는 함수

    Parameters:
        audio_path (str): 오디오 파일 경로
        n_fft (int): FFT(Fast Fourier Transform) 창의 크기 (default: 2048)
        hop_length (int): 이동 크기 (default: 512)
        n_mels (int): 멜 밴드의 수 (default: 128)

    Returns:
        mel_spectrogram (ndarray): 계산된 멜 스펙트럼
        sr (int): 오디오의 샘플링 레이트(초당 추출하는 샘플링 개수)
    """
    
    # 오디오 파일 불러오기 | audio.shape==(55680,) | sr=8000
    audio, sr = librosa.load(audio_path, sr=None)

    # 멜 스펙트럼 계산(y에 대한 멜 스펙트럼을 계산) | mel_spectrogram.shape=(128,109)
    mel_spectrogram = librosa.feature.melspectrogram(y=audio, sr=sr, n_fft=n_fft, hop_length=hop_length, n_mels=n_mels)

    # 로그 스케일 변환: 이를 통해 스펙트럼의 에너지를 더 쉽게 관찰하고, 시각화할 수 있다.
    # ref=np.max는 로그 변환 시 사용되는 기준값을 의미하며, 여기서는 스펙트럼 값 중 가장 큰 값으로 정규화한다.
    # mel_spectrogram.shape=(128,109)
    # 128이 세로이고 음석파일의 길이에 따라 가로 길이(두번째 숫자)가 길어지는 것 같다.
    mel_spectrogram = librosa.power_to_db(mel_spectrogram, ref=np.max)

    return mel_spectrogram, sr


def extract_features_and_labels(audio_files, class_labels):
    features, labels = [], []
    for file, label in zip(audio_files, class_labels):
        mel_spectrogram, _ = compute_mel_spectrogram(file)
        features.append(mel_spectrogram)
        labels.append(label)

    # [(128,109), (128, 109), (128, 108), (129, 110), ...]
    # max: (128, 109) | min: (128, 102)
    for i in range(len(features)):
        features[i] = np.array([feature[:102] for feature in features[i]])

    # 신경망 모델을 위해 데이터 변환
    features = np.array(features)
    labels = np.array(labels)

    return features, labels


X, y = extract_features_and_labels(audio_files, class_labels)

# 클래스 레이블 인코딩 및 원-핫 인코딩
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
num_classes = len(label_encoder.classes_)
y_one_hot = tf.keras.utils.to_categorical(y_encoded, num_classes=num_classes)

# 데이터 분할: 훈련 세트와 테스트 세트
X_train, X_test, y_train, y_test = train_test_split(X, y_one_hot, test_size=0.2, random_state=42)

/Users/jaewone/ENTER/envs/tf25/lib/python3.9/site-packages/numba/cpython/hashing.py:482: UserWarning: FNV hashing is not implemented in Numba. See PEP 456 https://www.python.org/dev/peps/pep-0456/ for rationale over not using FNV. Numba will continue to work, but hashes for built in types will be computed using siphash24. This will permit e.g. dictionaries to continue to behave as expected, however anything relying on the value of the hash opposed to hash as a derived property is likely to not work as expected.
  warnings.warn(msg)


In [12]:
# WaveNet model
class WaveNetClassifier():
    def __init__(self, input_shape, output_shape, kernel_size=2, dilation_depth=9, n_filters=40, task='classification', regression_range=None, load=False, load_dir='./'):
        """
        Parameters:
          input_shape: (tuple) tuple of input shape. (e.g. If input is 6s raw waveform with sampling rate = 16kHz, (96000,) is the input_shape)
          output_shape: (tuple)tuple of output shape. (e.g. If we want classify the signal into 100 classes, (100,) is the output_shape)
          kernel_size: (integer) kernel size of convolution operations in residual blocks
          dilation_depth: (integer) type total depth of residual blocks
          n_filters: (integer) # of filters of convolution operations in residual blocks
          task: (string) 'classification' or 'regression'
          regression_range: (list or tuple) target range of regression task
          load: (bool) load previous WaveNetClassifier or not
          load_dir: (string) the directory where the previous model exists
        """
        # save task info
        if task == 'regression':
            if regression_range[0] == 0:
                self.activation = 'sigmoid'
                self.scale_ratio = regression_range[1]
            elif regression_range[0] == - regression_range[1]:
                self.activation = 'tanh'
                self.scale_ratio = regression_range[1]
            elif regression_range == None:
                self.activation = 'linear'
                self.scale_ratio = 1
            else:
                print('ERROR: wrong regression range')
                sys.exit()
        elif task == 'classification':
            self.activation = 'softmax'
            self.scale_ratio = 1
        else:
            print('ERROR: wrong task')
            sys.exit()

        # save input info
        if len(input_shape) == 1:
            self.expand_dims = True
        elif len(input_shape) == 2:
            self.expand_dims = False
        else:
            print('ERROR: wrong input shape')
            sys.exit()
        self.input_shape = input_shape

        # save output info
        if len(output_shape) == 1:
            self.time_distributed = False
        elif len(output_shape) == 2:
            self.time_distributed = True
        else:
            print('ERROR: wrong output shape')
            sys.exit()
        self.output_shape = output_shape

        # save hyperparameters of WaveNet
        self.kernel_size = kernel_size
        self.dilation_depth = dilation_depth
        self.n_filters = n_filters
        self.manual_loss = None

        if load is True:
            self.model = load_model(
                load_dir+"saved_wavenet_clasifier.h5", custom_objects={'tf': tf})
            self.prev_history = pd.read_csv(
                load_dir+'wavenet_classifier_training_history.csv')
            self.start_idx = len(self.prev_history)
            self.history = None
        else:
            self.model = self.construct_model()
            self.start_idx = 0
            self.history = None
            self.prev_history = None

    def residual_block(self, x, i):
        tanh_out = Conv1D(self.n_filters,
                          self.kernel_size,
                          dilation_rate=self.kernel_size**i,
                          padding='causal',
                          name='dilated_conv_%d_tanh' % (
                              self.kernel_size ** i),
                          activation='tanh'
                          )(x)
        sigm_out = Conv1D(self.n_filters,
                          self.kernel_size,
                          dilation_rate=self.kernel_size**i,
                          padding='causal',
                          name='dilated_conv_%d_sigm' % (
                              self.kernel_size ** i),
                          activation='sigmoid'
                          )(x)
        z = Multiply(name='gated_activation_%d' % (i))([tanh_out, sigm_out])
        skip = Conv1D(self.n_filters, 1, name='skip_%d' % (i))(z)
        res = Add(name='residual_block_%d' % (i))([skip, x])
        return res, skip

    def construct_model(self):
        x = Input(shape=self.input_shape, name='original_input')
        if self.expand_dims == True:
            x_reshaped = Reshape(self.input_shape + (1,),
                                 name='reshaped_input')(x)
        else:
            x_reshaped = x
        skip_connections = []
        out = Conv1D(self.n_filters, 2, dilation_rate=1,
                     padding='causal', name='dilated_conv_1')(x_reshaped)
        for i in range(1, self.dilation_depth + 1):
            out, skip = self.residual_block(out, i)
            skip_connections.append(skip)
        out = Add(name='skip_connections')(skip_connections)
        out = Activation('relu')(out)
        out = Conv1D(self.n_filters, 80, strides=1, padding='same',
                     name='conv_5ms', activation='relu')(out)
        out = AveragePooling1D(
            80, padding='same', name='downsample_to_200Hz')(out)
        if self.time_distributed:
            # prev_len / x = target_len => x = prev_len / target_len
            target_kernel_size = (int)(
                self.input_shape[0] / 80 / self.output_shape[0])
            out = Conv1D(self.n_filters, target_kernel_size, padding='same',
                         name='conv_fit_to_target', activation='relu')(out)
            out = Conv1D(
                self.output_shape[1], target_kernel_size, padding='same', name='conv_final')(out)
            out = AveragePooling1D(target_kernel_size, padding='same')(out)
            out = TimeDistributed(Activation(self.activation))(out)
        else:
            out = Conv1D(self.n_filters, 100, padding='same',
                         activation='relu', name='conv_500ms')(out)
            out = Conv1D(self.output_shape[0], 100, padding='same',
                         activation='relu', name='conv_500ms_target_shape')(out)
            out = AveragePooling1D(100, padding='same',
                                   name='downsample_to_2Hz')(out)
            out = Conv1D(self.output_shape[0], (int)(
                self.input_shape[0] / 8000), padding='same', name='final_conv')(out)
            out = AveragePooling1D(
                (int)(self.input_shape[0] / 8000), name='final_pooling')(out)
            out = Reshape(self.output_shape)(out)
            out = Activation(self.activation)(out)
        if self.scale_ratio != 1:
            out = Lambda(lambda x: x * self.scale_ratio,
                         name='output_reshaped')(out)
        model = Model(x, out)
        model.summary()
        return model

    def get_model(self):
        return self.model

    def add_loss(self, loss):
        self.manual_loss = loss

    def fit(self, X, Y, validation_data=None, epochs=100, batch_size=32, optimizer='adam', save=False, save_dir='./'):
        # set default losses if not defined
        if self.manual_loss is not None:
            loss = self.manual_loss
            metrics = None
        else:
            if self.task == 'classification':
                loss = 'categorical_crossentropy'
                metrics = ['accuracy']
            else:
                loss = 'mean_squared_error'
                metrics = None

        # set callback functions
        if save:
            saved = save_dir + "saved_wavenet_clasifier.h5"
            hist = save_dir + 'wavenet_classifier_training_history.csv'
            if validation_data is None:
                checkpointer = ModelCheckpoint(
                    filepath=saved, monitor='loss', verbose=1, save_best_only=True)
            else:
                checkpointer = ModelCheckpoint(
                    filepath=saved, monitor='val_loss', verbose=1, save_best_only=True)
            history = History()
            callbacks = [history, checkpointer]
        else:
            callbacks = None

        # compile the model
        self.model.compile(optimizer, loss, metrics)
        try:
            self.history = self.model.fit(X, Y, shuffle=True, batch_size=batch_size, epochs=epochs,
                                          validation_data=validation_data, callbacks=callbacks, initial_epoch=self.start_idx)
        except:
            if save:
                df = pd.DataFrame.from_dict(history.history)
                df.to_csv(hist, encoding='utf-8', index=False)
            raise Exception('fit Error')
            sys.exit()
        return self.history

    def predict(self, x):
        return self.model.predict(x)

In [16]:
# Load model
model = WaveNetClassifier(
    input_shape = (16000*5,),
    output_shape = (7,),
    kernel_size=2,
    dilation_depth=9,
    n_filters=40,
    task='classification',
    regression_range=None,
    load=False,
)

Metal device set to: Apple M1

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2023-08-02 13:35:21.055236: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-08-02 13:35:21.055642: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 original_input (InputLayer)    [(None, 80000)]      0           []                               
                                                                                                  
 reshaped_input (Reshape)       (None, 80000, 1)     0           ['original_input[0][0]']         
                                                                                                  
 dilated_conv_1 (Conv1D)        (None, 80000, 40)    120         ['reshaped_input[0][0]']         
                                                                                                  
 dilated_conv_2_tanh (Conv1D)   (None, 80000, 40)    3240        ['dilated_conv_1[0][0]']         
                                                                                              